In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def load_train():
    folders = [x for x in os.listdir(data_path) if 'subset' in x]
    os.chdir(data_path)
    patients = []    
    for i in folders:
        os.chdir(data_path + i)
        patient_ids = [x for x in os.listdir(data_path + i) if '.mhd' in x]
        for id in patient_ids:
            j = '{}/{}'.format(i, id)
            patients.append(j)
    return patients

def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)
        
def normalize(image):
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image > 1] = 1.
    image[image < 0] = 0.
    return image

def voxel_2_world(voxel_coordinates, origin, spacing):
    stretched_voxel_coordinates = voxel_coordinates * spacing
    world_coordinates = stretched_voxel_coordinates + origin
    return world_coordinates

def pred_samples(img_file):
    patient_id = img_file.split('/')[-1][:-4]
    #读取mhd
    itk_img = SimpleITK.ReadImage(data_path + patient) 
    img_array = SimpleITK.GetArrayFromImage(itk_img) # inde
    origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
    spacing = np.array(itk_img.GetSpacing())
    direction = np.array(itk_img.GetDirection())
    if round(direction[0]) != 1:
        origin[0] *= -1
        print patient_id
    if round(direction[4]) != 1:
        origin[1] *= -1   
        print patient_id
    
    #加上肺部掩膜后进行预测（先加和后加）
    if img_array.shape[1] != 512:
        print patient_id
    else:
        img_array_new = np.zeros_like(img_array)
        for i in tqdm(range(img_array.shape[0])):
            img = img_array[i]
            #seg_img, overlap = helpers.get_segmented_lungs(img.copy())
            img = normalize(img) * 255        
            img = np.expand_dims(img,0)
            img = np.expand_dims(img,0)
            p = model.predict(img)
            #p = p*overlap
            img_array_new[i] = p 
    return img_array_new, patient_id,origin, spacing

def get_coord(img_array_new, patient_id,origin, spacing):
    var = []
#提取连通区域                     
    temp = np.squeeze(img_array_new)


    temp = skimage.morphology.binary_opening(np.squeeze(temp), np.ones([5,5,5]))

    labels = skimage.measure.label(np.squeeze(temp))    
    props = skimage.measure.regionprops(labels)
    for i in range(len(props)):
        #if props[i]['EquivDiameter'] < 100:        
        world_coordinates = voxel_2_world([props[i]['Centroid'][2], 
                                                   props[i]['Centroid'][1], 
                                                   props[i]['Centroid'][0]], origin, spacing)               
        var.append([patient_id,
                    world_coordinates[0],
                    world_coordinates[1],
                    world_coordinates[2],
                    props[i]['EquivDiameter']])        
    return var

In [3]:
csv_path = PATH['annotations_train']
model_paths = PATH['model_paths']
pred_path = PATH['pic_train']
data_path = PATH['src_train']

In [4]:
model_fenge_path=model_paths + 'final_fenge_170607_2.h5'
model = load_model(model_fenge_path, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

In [5]:
patients = load_train()
anno = pd.read_csv('/Volumes/solo/ali/csv/train/annotations.csv')

In [6]:
patient = patients[10]
patient

'train_subset00/LKDS-00020.mhd'

In [7]:
#for patient in sorted(patients):
img_pred, patient_id, origin, spacing = pred_samples(patient)

100%|██████████| 221/221 [00:26<00:00,  8.71it/s]


In [8]:
var = get_coord(img_pred, patient_id,origin, spacing)

In [9]:
var_csv = pd.DataFrame.from_records(var,columns=['seriesuid','coordX','coordY','coordZ','diameter_mm'])

In [10]:
def cal_dist(var_csv,anno):
    dist = []
    ratio = []
    for index_csv,row_csv in var_csv.iterrows():
        mini_ann = anno[anno['seriesuid']==row_csv['seriesuid']] 
        dist_ = []
        ratio_ = []
        for index_ann,row_ann in mini_ann.iterrows():        
            vec1 = np.array([row_csv['coordX'],row_csv['coordY'],row_csv['coordZ']])
            vec2 = np.array([row_ann['coordX'],row_ann['coordY'],row_ann['coordZ']])
            ratio_.append(numpy.linalg.norm(vec1 - vec2)/row_ann['diameter_mm'])
            dist_.append(numpy.linalg.norm(vec1 - vec2))
        ratio.append(np.min(ratio_))
        dist.append(np.min(dist_))  
    var_csv['ratio'] = ratio
    var_csv['dist'] = dist
    var_csv = var_csv.sort_values('dist')
    return var_csv

In [11]:
final = cal_dist(var_csv,anno)
pd.set_option('display.max_rows', 200)
final

,seriesuid,coordX,coordY,coordZ,diameter_mm,ratio,dist
1,LKDS-00020,-70.185698,11.648193,48.339697,30.142794,0.236906,5.301128
0,LKDS-00020,-21.633460,31.431986,22.869898,13.786430,2.632340,58.902602
2,LKDS-00020,161.916070,-15.689607,229.444103,9.194581,13.418925,83.544791
